# Functions

## obj_prepare_path

In [17]:
def obj_prepare_path( obj, salt, status )
  def helper_parse_path( str )
    if str[ str.length - 1, 1 ] != '/'
      str = str + '/'
    end
    return str
  end

  def draw_obj_line_edge( l, offset )
      str = ''
      for i in 1..( ( l - 1 ) * offset )
        str += ' '
      end
      if l > 1
        str += "┗"
        str += "━"
        str += " "
      else
        str += '    '
      end
      return str
  end
  
  def draw_obj_path_local( str, l, offset )
    result = ''
    for i in 0..( ( l - 1 ) * offset )
      result += ' '
    end
    result += ''
    result += str
    return result
  end

  def helper_obj_path( name, salt, k, f=nil )
    str = ''
    str += draw_obj_line_edge( name[ 1, name.length ].to_i, 4 )
    str += 'hash[:path]'
    if k.length == 0

    else
      for i in 0..k.length-1
        str += '[:children]'
        str += '[:'
        str += k[ i ].to_s
        str += ']'      
      end
    end

    if !f.nil?
      str += '[:files][:'
      str += f.to_s
      str += '][:full]'    
    else
      str += '[:full]'
    end
    return str
  end
  
  def helper_insert_salt( salt, str )
    if salt[ 0, 1 ] == ''
      
    else
      if salt[ 0, 1 ] == '-'
      else
        salt = '-' + salt      
      end
    end

    str = str.gsub( "{{SALT}}", salt )
    return str
  end

  def helper_parse_path( str )
    if str[ str.length - 1, 1 ] != '/'
      str = str + '/'
    end
    return str
  end

  def draw_obj_line_edge( l, offset )
      str = ''
      for i in 1..( ( l - 1 ) * offset )
        str += ' '
      end
      if l > 1
        str += "┗"
        str += "━"
        str += " "
      else
        str += '    '
      end
      return str
  end
  
  def draw_obj_path_local( str, l, offset )
    result = ''
    for i in 0..( ( l - 1 ) * offset )
      result += ' '
    end
    result = result[ 1, result.length ]
    
    if str.index('.') != nil
      result += 'File: '
    else
      result += 'Folder: '
    end
    result += str
    return result
  end

  def helper_obj_path( name, salt, k, f=nil )
    str = ''
    str += draw_obj_line_edge( name[ 1, name.length ].to_i, 4 )
    str += ''
    str += 'hash[:path]'
    if k.length == 0

    else
      for i in 0..k.length-1
        str += '[:children]'
        str += '[:'
        str += k[ i ].to_s
        str += ']'      
      end
    end

    if !f.nil?
      str += '[:files][:'
      str += f.to_s
      str += '][:full]'    
    else
      str += '[:full]'
    end
    return str
  end
  
  def helper_insert_salt( salt, str )
    if salt[ 0, 1 ] == ''
      
    else
      if salt[ 0, 1 ] == '-'
      else
        salt = '-' + salt      
      end
    end

    str = str.gsub( "{{SALT}}", salt )
    return str
  end

  mode = {
    :general => nil,
    :hash => nil,
    :path => nil
  }
    
  case status
    when 0
      mode[:general] = false
      mode[:hash] = false
      mode[:path] = false
    when 1
      mode[:general] = true
      mode[:hash] = true
      mode[:path] = false
    when 2
      mode[:general] = true
      mode[:hash] = false
      mode[:path] = true
    when 3
      mode[:general] = true
      mode[:hash] = true
      mode[:path] = true
  end
  
  
  mode[:general] ? puts( 'TREE OVERVIEW' ) : ''
  
  obj[:full] = ''
  obj[:full] += obj[:root]
  obj[:full] += helper_parse_path( helper_insert_salt( salt, obj[:name] ) )
  
  mode[:hash] ? puts( helper_obj_path( 'l1', salt, [ ] ) ) : ''
  mode[:path] ? puts( draw_obj_path_local( obj[:full], 2, 4 ) ) : ''

  obj[:children].keys.each { | l2 | 
      obj[:children][ l2 ][:full] = ''
      obj[:children][ l2 ][:full] += obj[:full]
      obj[:children][ l2 ][:full] += helper_parse_path( helper_insert_salt( salt, obj[:children][ l2 ][:name] ) )
      mode[:hash] ? puts( helper_obj_path( 'l2', salt, [ l2 ] ) ) : ''
      mode[:path] ? puts( draw_obj_path_local( obj[:children][ l2 ][:full], 3, 4 ) ) : ''
      FileUtils.mkdir_p obj[:children][ l2 ][:full]

      if !obj[:children][ l2 ][:files].nil?
        obj[:children][ l2 ][:files].keys.each { | f1 | 
          obj[:children][ l2 ][:files][ f1 ][:full] = '' 
          obj[:children][ l2 ][:files][ f1 ][:full] += obj[:children][ l2 ][:full] 
          obj[:children][ l2 ][:files][ f1 ][:full] += helper_insert_salt( salt, obj[:children][ l2 ][:files][ f1 ][:name] )
          mode[:hash] ? puts( helper_obj_path( 'l2', salt, [ l2 ], f1 ) ) : ''
          mode[:path] ? puts( draw_obj_path_local( obj[:children][ l2 ][:files][ f1 ][:full], 3, 4 ) ) : ''
        }
      end

      if !obj[:children][ l2 ][:children].nil?
        obj[:children][ l2 ][:children].keys.each { | l3 | 
          obj[:children][ l2 ][:children][ l3 ][:full] = ''
          obj[:children][ l2 ][:children][ l3 ][:full] += obj[:children][ l2 ][:full]
          obj[:children][ l2 ][:children][ l3 ][:full] += helper_parse_path( helper_insert_salt( salt, obj[:children][ l2 ][:children][ l3 ][:name] ) )

          FileUtils.mkdir_p obj[:children][ l2 ][:children][ l3 ][:full]
          mode[:hash] ? puts( helper_obj_path( 'l3', salt, [ l2, l3 ] ) ) : ''
          mode[:path] ? puts( draw_obj_path_local( obj[:children][ l2 ][:children][ l3 ][:full], 4, 4 ) ) : ''

          if !obj[:children][ l2 ][:children][ l3 ][:files].nil?
            obj[:children][ l2 ][:children][ l3 ][:files].keys.each { | f2 | 
              obj[:children][ l2 ][:children][ l3 ][:files][ f2 ][:full] = ''
              obj[:children][ l2 ][:children][ l3 ][:files][ f2 ][:full] += obj[:children][ l2 ][:children][ l3 ][:full]
              obj[:children][ l2 ][:children][ l3 ][:files][ f2 ][:full] += helper_insert_salt( salt, obj[:children][ l2 ][:children][ l3 ][:files][ f2 ][:name] )
              mode[:hash] ? puts( helper_obj_path( 'l3', salt, [ l2, l3 ], f2 ) ) : ''
              mode[:path] ? puts( draw_obj_path_local( obj[:children][ l2 ][:children][ l3 ][:files][ f2 ][:full], 4, 4 ) ) : ''
            }
          end
        }
      end
  }
  return obj
end

:obj_prepare_path

## explore_hash

In [103]:
def explore_hash( obj, debug )
    def draw_hash_line_edge( l, offset )
        str = ''
        l > 1
            for i in 1..( ( l - 1 ) * offset )
            str += ' '
            end
            if l > 1
            str += "┗"
            str += "━"
            str += " "
            else
            str += '    '
            end
        return str
    end

    def draw_hash_line( name, offset, key )
        level = name[ 1, name.length ].to_i
        result = ''
        result += draw_hash_line_edge( level, offset )
        result += key.to_s
        result += ' ('
        result += level.to_s
        #result += ' '
        #result += key.class.to_s
        result += ')'
        return result
    end
        
    struct = obj
    types =  [ Hash, ActiveSupport::HashWithIndifferentAccess ]
    offset = 4
    
    puts 'TREE OVERVIEW'
    struct.keys.each { | l1 | 
        puts draw_hash_line( 'l1', offset, l1 )
        if types.include? struct[ l1 ].class
            struct[ l1 ].keys.each { | l2 |
                puts draw_hash_line( 'l2', offset, l2 )
                if types.include? struct[ l1 ][ l2 ].class
                    struct[ l1 ][ l2 ].keys.each { | l3 |
                        puts draw_hash_line( 'l3', offset, l3 )
                        if types.include? struct[ l1 ][ l2 ][ l3 ].class
                            struct[ l1 ][ l2 ][ l3 ].keys.each { | l4 | 
                                puts draw_hash_line( 'l4', offset, l4 )
                                if types.include? struct[ l1 ][ l2 ][ l3 ][ l4 ].class
                                    struct[ l1 ][ l2 ][ l3 ][ l4 ].keys.each { | l5 | 
                                        puts draw_hash_line( 'l5', offset, l5 )
                                        if types.include? struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ].class
                                            struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ].keys.each { | l6 | 
                                                puts draw_hash_line( 'l6', offset, l6 )
                                                if types.include? struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ].class
                                                    struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ].keys.each { | l7 | 
                                                        puts draw_hash_line( 'l7', offset, l7 )
                                                        if types.include? struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ].class
                                                            struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ].keys.each { | l8 | 
                                                                puts draw_hash_line( 'l8', offset, l8 )
                                                                if types.include? struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ].class
                                                                    struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ].keys.each { | l9 | 
                                                                        puts draw_hash_line( 'l9', offset, l9 )
                                                                        if types.include? struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ][ l9 ].class
                                                                            struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ][ l9 ].keys.each { | l10 | 
                                                                                puts draw_hash_line( 'l10', offset, l10 )
                                                                                if types.include? struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ][ l9 ][ l10 ].class
                                                                                    struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ][ l9 ][ l10 ].keys.each { | l11 | 
                                                                                        puts draw_hash_line( 'l11', offset, l11 )
                                                                                        if types.include? struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ][ l9 ][ l10 ][ l11 ].class
                                                                                            struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ][ l9 ][ l10 ][ l11 ].keys.each { | l12 | 
                                                                                                puts draw_hash_line( 'l12', offset, l12 )
                                                                                                if types.include? struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ][ l9 ][ l10 ][ l11 ][ l12 ].class
                                                                                                    struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ][ l9 ][ l10 ][ l11 ][ l12 ].keys.each { | l13 | 
                                                                                                        puts draw_hash_line( 'l13', offset, l13 )
                                                                                                        if types.include? struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ][ l9 ][ l10 ][ l11 ][ l12 ][ l13 ].class
                                                                                                            struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ][ l9 ][ l10 ][ l11 ][ l12 ][ l13 ].keys.each { | l14 | 
                                                                                                                puts draw_hash_line( 'l14', offset, l14 )
                                                                                                                if types.include? struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ][ l9 ][ l10 ][ l11 ][ l12 ][ l13 ][ l14 ].class
                                                                                                                    struct[ l1 ][ l2 ][ l3 ][ l4 ][ l5 ][ l6 ][ l7 ][ l8 ][ l9 ][ l10 ][ l11 ][ l12 ][ l13 ][ l14 ].keys.each { | l15 | 
                                                                                                                        puts draw_hash_line( 'l15', offset, l15 )
                                                                                                                        puts 'MORE....'
                                                        
                                                                                                                    }
                                                                                                                end
                                                                                                            }
                                                                                                        end
                                                                                                    }
                                                                                                end
                                                                                            }
                                                                                        end
                                                                                    }
                                                                                end
                                                                            }
                                                                        end
                                                                    }
                                                                end
                                                            }
                                                        end
                                                    }
                                                end
                                            }
                                        end
                                    }
                                end
                            }
                        end
                    }
                end
            }
        end
    }
end

:explore_hash

## yt_get_params

In [178]:
def yt_get_params( video_id, debug )
  def download_html( url, debug )
    uri = URI( url )
    res = Net::HTTP.get( uri ) 
    html = Nokogiri::HTML( res )
    return html
  end

  def download_xml( url, debug )
    uri = URI( url )
    res = Net::HTTP.get( uri ) 
    xml = Nokogiri::XML( res )
    return xml
  end
  
  def parse_params( html )
    select = {
      script: {
        xpath: '//script',
        selector: 'var ytcfg'
      },
      params: {
        from: {
          gsub: "ytcfg.set(",
          index: nil
        },
        to: {
          gsub: '});',
          index: nil
        }    
      }
    }

    scripts = html.xpath( select[:script][:xpath] )
    script = scripts.find { | s | s.text.start_with?( select[:script][:selector] ) }

    tmp = script.text
    select[:params][:from][:index] = tmp.index( select[:params][:from][:gsub] )
    tmp = tmp[ select[:params][:from][:index] + select[:params][:from][:gsub].length, tmp.length]
    select[:params][:to][:index] = tmp.index( select[:params][:to][:gsub] )
    tmp = tmp[ 0, select[:params][:to][:index] ]
    tmp << select[:params][:to][:gsub][ 0, 1 ]
    params = JSON.parse( tmp )
    data = { :data => params }
    data = data.with_indifferent_access
    data[:data][:PLAYER_VARS][:embedded_player_response] = JSON.parse( data[:data][:PLAYER_VARS][:embedded_player_response] )
    data = data.with_indifferent_access
    return data
  end
  
  def parse_channel_id( data, keys )
    tmp = {}    
    tmp = data[:data]
    keys.each { | key | tmp = tmp[ key ]}
    channel_id = tmp
    return channel_id
  end

  item = {
    :video => {
      :id => video_id,
      :duration => nil
    },
    :channel => {
      :id => nil,
      :name => nil
    }
  }
  
  sources = { :embed => 'https://www.youtube.com/embed/' }

  url = ''
  url += sources[:embed]
  url += item[:video][:id]

  html = download_html( url, debug )
  data = parse_params( html )
  
  
  keys = [
    :PLAYER_VARS,
    :embedded_player_response,
    :embedPreview,
    :thumbnailPreviewRenderer,
    :videoDetails,
    :embeddedPlayerOverlayVideoDetailsRenderer,
    :expandedRenderer,
    :embeddedPlayerOverlayVideoDetailsExpandedRenderer,
    :subscribeButton,
    :subscribeButtonRenderer,
    :channelId
  ]
  
  item[:channel][:id] = parse_channel_id( data, keys )
  
  if !item[:channel][:id].nil?
    puts feed = 'https://www.youtube.com/feeds/videos.xml?channel_id=' + item[:channel][:id]
    xml = download_xml( feed, debug )
    item[:channel][:name] = xml.at( 'feed' ).css( 'title' )[ 0 ].text
  end

  
  return item
end

:yt_get_params

# Execute

In [97]:
require 'nokogiri'
require 'sinatra'
require 'json'
require 'net/http'
require 'active_support/core_ext/hash/indifferent_access'

true

In [106]:
hash = {
  path: {
    root: './',
    name: 'test',
    children: {
      html: {
        name: 'html',
        full: nil,
        files: {
          html: {
            name: 'yt-1.html'
          },
          json: {
            name: 'yt-1.json'
          }
        }
      }
    }
  }
}

hash[:path] = obj_prepare_path( hash[:path], '', 1 )
puts

TREE OVERVIEW
    hash[:path][:full]
    ┗━ hash[:path][:children][:html][:full]
    ┗━ hash[:path][:children][:html][:files][:html][:full]
    ┗━ hash[:path][:children][:html][:files][:json][:full]



## Explore Params

In [179]:
data = yt_get_params( 'd8AW2XS8obU', true )
#path = hash[:path][:children][:html][:files][:json][:full]
#File.open( path , "w" ) { | f | f.write( JSON.pretty_generate( data ) ) }
#explore_hash( data, true)

https://www.youtube.com/feeds/videos.xml?channel_id=UCHpymKKyZTz9qKYK1GvBA3g


{:video=>{:id=>"d8AW2XS8obU", :duration=>nil}, :channel=>{:id=>"UCHpymKKyZTz9qKYK1GvBA3g", :name=>"Depressionshilfe"}}

In [173]:
channel_id = data[:data][:PLAYER_VARS][:embedded_player_response][:embedPreview][:thumbnailPreviewRenderer][:videoDetails][:embeddedPlayerOverlayVideoDetailsRenderer][:expandedRenderer][:embeddedPlayerOverlayVideoDetailsExpandedRenderer][:subscribeButton][:subscribeButtonRenderer][:channelId]



"UCHpymKKyZTz9qKYK1GvBA3g"

"UCHpymKKyZTz9qKYK1GvBA3g"